In [2]:
# imports
import os, rasterio
import time
from deepforest import main, utilities, get_data, preprocess 
from os.path import exists
from  PIL import Image, ImagePath
Image.MAX_IMAGE_PIXELS = None
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# get and check Image
image_path=r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\PNG\DQ_2022_3b.png"
# "D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\PNG\DQ_2022_3b.png"
print(exists(image_path))

# annotations = ("D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\annotation\\deep-forest-training.csv")
# print(exists(annotations))

exImg=Image.open(image_path)
print(exImg.getbands())
print(exImg.size)
src = rasterio.open(image_path)

transform = src.transform 
crs = src.crs
print(crs)

True
('R', 'G', 'B')
(15652, 11592)
EPSG:32638


In [4]:
# get training bonding box and convert to the right format
ano_shp=r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\DQ-Vegetation-Analysis-1180\03_Data\model-training\20230121-Five-classes-data-v1\1class-treesdata-start-December2022.shp"
annotations=utilities.shapefile_to_annotations(ano_shp, image_path,  savedir='.')

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_HDF4.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_HDF4.dll
127: The specified procedure could not be found.

Can't load r

In [5]:
annotations.to_csv(r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\DQ-Vegetation-Analysis-1180\03_Data\model-training\20230121-Five-classes-data-v1\1class-treesdata-start-December2022.csv")

In [6]:
ano_csv=r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\DQ-Vegetation-Analysis-1180\03_Data\model-training\20230121-Five-classes-data-v1\1class-treesdata-start-December2022.csv"

In [7]:
#Write csv to file and crop
tmpdir = (r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\crop")
# annotations.to_csv("{}/example.csv".format(tmpdir), index=False)
annotations_file = preprocess.split_raster(path_to_raster=image_path,
                                           annotations_file=ano_csv,
                                           base_dir=tmpdir,
                                           patch_size=500,
                                           patch_overlap=0.25)

# Returns a 6 column pandas array

c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\preprocess.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_annotations.image_path = "{}_{}.png".format(image_basename, index)
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\preprocess.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_annotations.xmax = (selected_annotations.xmin - window_xmin) + (
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\preprocess.py:112: SettingWithCopyWarning

In [9]:
del(annotations_file['Unnamed: 0'])
assert annotations_file.shape[1] == 6

In [10]:
annotations_file

,image_path,xmin,ymin,xmax,ymax,label
2453,DQ_2022_3b_322.png,473,450,496,473,Tree
2148,DQ_2022_3b_323.png,414,403,426,415,Tree
2161,DQ_2022_3b_323.png,459,457,500,499,Tree
2244,DQ_2022_3b_323.png,408,368,413,373,Tree
2245,DQ_2022_3b_323.png,420,355,423,358,Tree
...,...,...,...,...,...,...
1274,DQ_2022_3b_919.png,5,253,10,259,Tree
1275,DQ_2022_3b_919.png,32,103,43,114,Tree
1299,DQ_2022_3b_919.png,42,138,53,149,Tree
1300,DQ_2022_3b_919.png,53,128,55,130,Tree


In [16]:
ano_split=get_data(r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\crop\DQ_2022_3b.csv")
exists(ano_split)

True

In [15]:
# pd.DataFrame("D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\crop\DQ_2022_3b.csv")
# os.path.dirname(ano_split)

ValueError: DataFrame constructor not properly called!

In [17]:
model = main.deepforest()
model.use_release()

# Example run with short training
model.config["train"]['epochs'] = 5
model.config['gpus'] = '-1' #move to GPU and use all the GPU resources
# model.config['cpu'] = 'accelerator'
model.config["save-snapshot"] = False
model.config["train"]["csv_file"] = ano_split
model.config["train"]["root_dir"] = os.path.dirname(ano_split)
model.config['num_worker']=16
# model.config['max_epochs']=10
model.create_trainer()



Reading config file: c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\data\deepforest_config.yml


c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus='-1')` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices='-1')` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True


Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/1.0.0 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
start_time = time.time()
model.trainer.fit(model)
print(f"--- Training on CPU: {(time.time() - start_time):.2f} seconds ---")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | RetinaNet | 32.1 M
------------------------------------
31.9 M    Trainable params
222 K     Non-trainable params
32.1 M    Total params
128.592   Total estimated model params size (MB)


c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:366: UserWarning: One of given dataloaders is None and it will be skipped.
  rank_zero_warn("One of given dataloaders is None and it will be skipped.")
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 2:   4%|▎         | 9/249 [00:05<02:33,  1.57it/s, loss=2.06]  

In [ ]:
# boxes = model.predict_image(path=image_path, return_plot = True)
predicted_raster = model.predict_tile(image_path, return_plot = False, patch_size=500,patch_overlap=0.25)

In [ ]:
pd.DataFrame(predicted_raster)

In [ ]:
r = rasterio.open(image_path)
transform = r.transform 
crs = r.crs
print(crs)

gdf = utilities.annotations_to_shapefile(predicted_raster, transform=transform, crs=crs)

In [ ]:
gdf.to_file('trained-500-1e-230.shp')

In [ ]:
results = model.evaluate(ano_split, os.path.dirname(ano_split), iou_threshold = 0.45)

In [ ]:
print(results["box_recall"])
print(results["box_precision"])

In [ ]:
results["results"].head()

In [ ]:
# Importing Image class from PIL module
 
# Opens a image in RGB mode
RUH_img = Image.open(r"\\?\D:\Dropbox\P.Robeck\BPLA Dropbox\08 GIS-data\GIS-data-Riyadh\2022-03-world-view-3\RCRC-4Bands-Mosaic_FIN_I542628_614813\RCRC-4Bands-Mosaic_FIN_I542628_614813\RCRC-4Bands-Mosaic_I542628_FL01-P244705\RCRC-4Bands-Mosaic_MOS\22FEB15073634-S2AM-RCRC-4Bands-Mosaic.TIF")
DQ_img = Image.open(r"D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\PNG\\DQ_2022_3b.png")

In [ ]:
DQ_img.getbbox()

In [ ]:
print(DQ_img.format, DQ_img.size, DQ_img.mode,DQ_img.getbbox)

In [ ]:
# test GPU
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer   
  
# normal function to run on cpu
def func(a):                                
    for i in range(10000000):
        a[i]+= 1      
  
# function optimized to run on gpu 
@jit(target_backend='cuda')                         
def func2(a):
    for i in range(10000000):
        a[i]+= 1
if __name__=="__main__":
    n = 10000000                            
    a = np.ones(n, dtype = np.float64)
      
    start = timer()
    func(a)
    print("without GPU:", timer()-start)    
      
    start = timer()
    func2(a)
    print("with GPU:", timer()-start)